In [346]:
import torch
import random
import argparse
import pandas as pd
import utils
from transformers import BertTokenizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from loss_functions import LabelSmoothingLoss

random.seed(0)

from model import BertBaselineClassifier

In [347]:
# Save the device
device = torch.cuda.current_device() if torch.cuda.is_available() else 'cpu'

# Initialize the model
model = BertBaselineClassifier(weights_name='bert-base-cased',
                               n_classes_=2,
                               batch_size=4)


In [348]:
# Initialize data
data = pd.read_csv(
    'C:/Users/xiuyu/nlu/data/AgreementDataset/agreement_dataset_valid_tweets.tsv', sep='\t')
X = data['text']
y = data['label']
confidence = data['agreement_level']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [349]:
X_train

6443    .@JoeBiden included images of Kyle Rittenhouse...
1620    @therecount @stefcutter @realDonaldTrump as a ...
2820      He is another nut case. https://t.co/UeBcOGTmYL
1643    @JoeBiden White supremacists are still a thoud...
2877    @flinch04 @FOX13News It's well wishes. In a cr...
                              ...                        
3772         CRAZY is contagious! https://t.co/k2qcuu9YEj
5191    RT @CharlesDonnor: Hell we gave em smallpox an...
5226    RT @BenandBrackenRu: Simple, clear and easily ...
5390    We only have one chance, and that is November ...
860     @Rannirebel @RealLucyLawless Savannah confused...
Name: text, Length: 5453, dtype: object

In [350]:
y_train

6443    0
1620    1
2820    1
1643    0
2877    0
       ..
3772    0
5191    1
5226    0
5390    0
860     1
Name: label, Length: 5453, dtype: int64

In [351]:
y = data['label']
y

0       1
1       0
2       0
3       0
4       1
       ..
6812    0
6813    0
6814    0
6815    0
6816    1
Name: label, Length: 6817, dtype: int64

In [352]:
confidence

0       A++
1       A++
2        A+
3       A++
4       A++
       ... 
6812    A++
6813     A+
6814     A+
6815    A++
6816    A++
Name: agreement_level, Length: 6817, dtype: object

In [39]:
# train
smoothed_labels = utils.smooth_labels(y, confidence)
model.fit(X_train, y_train)

torch.Size([5453])
torch.Size([5453, 327])


Stopping after epoch 16. Training loss did not improve more than tol=1e-05. Final error is 821.289023399353.

BertBaselineClassifier(
	batch_size=4,
	max_iter=1000,
	eta=0.001,
	optimizer_class=<class 'torch.optim.adam.Adam'>,
	l2_strength=0,
	gradient_accumulation_steps=1,
	max_grad_norm=None,
	validation_fraction=0.1,
	early_stopping=False,
	n_iter_no_change=10,
	warm_start=False,
	tol=1e-05,
	hidden_dim=50,
	hidden_activation=Tanh())

In [40]:
predictions = model.predict(X_test)
report = pd.DataFrame(classification_report(y_test, predictions, digits=3, output_dict=True)).transpose()

D:\Users\xiuyu\anaconda3\envs\nlu\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Users\xiuyu\anaconda3\envs\nlu\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Users\xiuyu\anaconda3\envs\nlu\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [41]:
report

,precision,recall,f1-score,support
0,0.715543,1.000000,0.834188,976.000000
1,0.000000,0.000000,0.000000,388.000000
accuracy,0.715543,0.715543,0.715543,0.715543
macro avg,0.357771,0.500000,0.417094,1364.000000
weighted avg,0.512001,0.715543,0.596897,1364.000000


In [298]:
import torch.nn as nn
from transformers import RobertaTokenizer, RobertaModel,RobertaConfig

In [303]:
class RobertaModel_(nn.Module):
    config = RobertaConfig()
    def __init__(self, weights_name, n_classes_, *args, **kwargs):
        super().__init__()
        self.roberta = RobertaModel.from_pretrained(weights_name, num_labels= n_classes_ ,output_hidden_states=True,output_attentions=False)
        self.pre_classifier = torch.nn.Linear(768, 768) 
        self.dropout = torch.nn.Dropout(0.225)
        self.classifier = torch.nn.Linear(768, 5)

        
    def forward(self, input_ids, attention_mask):
        x = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = x[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        return self.classifier(pooler)     
        


In [304]:
class RobertaClassifier(TorchShallowNeuralClassifier):

    def __init__(self, weights_name, n_classes_, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.weights_name = weights_name
        self.n_classes_ = n_classes_
        self.tokenizer = RobertaTokenizer.from_pretrained(weights_name)

    def build_graph(self):
        # computation graph
        # sets model in TorchShallowNeuralClassifier fit
        return RobertaModel_(self.weights_name, self.n_classes_)

# TODO: modify to use empirical distribution as labels as in Ex Machina: Personal Attacks Seen at Scale?
    def build_dataset(self, X, y=None):
        data = self.tokenizer.batch_encode_plus(
            X,
            max_length=None,
            add_special_tokens=True,
            padding='longest',
            return_attention_mask=True)
        indices = torch.tensor(data['input_ids'])
        mask = torch.tensor(data['attention_mask'])
        if y is None:
            dataset = torch.utils.data.TensorDataset(indices, mask)
        else:
            self.classes_ = sorted(set(y))
            self.n_classes_ = len(self.classes_)
            class2index = dict(zip(self.classes_, range(self.n_classes_)))
            y = [class2index[label] for label in y]
            y = torch.tensor(y)
            print(y.size())
            print(indices.size())
            dataset = torch.utils.data.TensorDataset(indices, mask, y)
        return dataset


    # TODO: Implement score() in case we adopt metrics proposed in http://www.kayur.org/papers/chi2021.pdf
    def score(self, X, y, device=None):
        """
        Uses macro-F1 as the score function.

        This function can be used to evaluate models, but its primary
        use is in cross-validation and hyperparameter tuning.

        Parameters
        ----------
        X: np.array, shape `(n_examples, n_features)`

        y: iterable, shape `len(n_examples)`
            These can be the raw labels. They will converted internally
            as needed. See `build_dataset`.

        device: str or None
            Allows the user to temporarily change the device used
            during prediction. This is useful if predictions require a
            lot of memory and so are better done on the CPU. After
            prediction is done, the model is returned to `self.device`.

        Returns
        -------
        float

        """
        preds = self.predict(X, device=device)
        return utils.safe_macro_f1(y, preds)

In [305]:
# Initialize the model
model_2 = RobertaClassifier(weights_name='roberta-base',
                               n_classes_=2,
                               batch_size=2)

In [306]:
# train
model_2.fit(X_train, y_train)

torch.Size([5453])
torch.Size([5453, 282])


Stopping after epoch 13. Training loss did not improve more than tol=1e-05. Final error is 3255.478422820568.

RobertaClassifier(
	batch_size=2,
	max_iter=1000,
	eta=0.001,
	optimizer_class=<class 'torch.optim.adam.Adam'>,
	l2_strength=0,
	gradient_accumulation_steps=1,
	max_grad_norm=None,
	validation_fraction=0.1,
	early_stopping=False,
	n_iter_no_change=10,
	warm_start=False,
	tol=1e-05,
	hidden_dim=50,
	hidden_activation=Tanh())

In [311]:
model_2.to_pickle('C:/Users/xiuyu/nlu/roberta.pkl')

In [334]:
predictions_2 = model_2.predict(X_test)

In [335]:
from sklearn.metrics import classification_report
report_2 = pd.DataFrame(classification_report(y_test, predictions_2, digits=3, output_dict=True)).transpose()

D:\Users\xiuyu\anaconda3\envs\nlu\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Users\xiuyu\anaconda3\envs\nlu\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Users\xiuyu\anaconda3\envs\nlu\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [355]:
report_2

,precision,recall,f1-score,support
0,0.715543,1.000000,0.834188,976.000000
1,0.000000,0.000000,0.000000,388.000000
accuracy,0.715543,0.715543,0.715543,0.715543
macro avg,0.357771,0.500000,0.417094,1364.000000
weighted avg,0.512001,0.715543,0.596897,1364.000000
